***GENERATED CODE FOR bceligdenialsdayspred PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import warnings
warnings.filterwarnings('ignore')


class RDBMSConnector:

    def fetch(spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        return spark.read.format("jdbc") \
            .option("url", f"jdbc:sqlserver://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option(eval(config)['qtype'], eval(config)['query']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .load()

    def put(df, spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        # Write modes: overwrite, append
        df.write.mode(eval(config)['writemode'])\
            .format('jdbc') \
            .option("url", f"jdbc:{eval(config)['dbtype']}://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option("dbtable", eval(config)['table']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .save()


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = transformationDF.drop('ChargeDetailID')
        transformationDF = transformationDF.drop('DateofService')
        transformationDF = transformationDF.drop('TransactionDate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CPTCode', 'transformation_label': 'String Indexer'}], 'feature': 'CPTCode', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '12499', 'mean': '145673.42', 'stddev': '708211.55', 'min': '10060', 'max': 'S9083', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CPTCode'}, {'feature_label': 'CPTCode', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CPTCode')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CptModifier', 'transformation_label': 'String Indexer'}], 'feature': 'CptModifier', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '12499', 'mean': '8.92', 'stddev': '19.43', 'min': '0000', 'max': 'XS', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CptModifier'}, {'feature_label': 'CptModifier', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CptModifier')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CptModifier2', 'transformation_label': 'String Indexer'}], 'feature': 'CptModifier2', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '12499', 'mean': '0.02', 'stddev': '1.03', 'min': '0000', 'max': 'RT', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CptModifier2'}, {'feature_label': 'CptModifier2', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CptModifier2')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CptModifier3', 'transformation_label': 'String Indexer'}], 'feature': 'CptModifier3', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '12499', 'mean': '0.0', 'stddev': '0.0', 'min': '0000', 'max': 'RT', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CptModifier3'}, {'feature_label': 'CptModifier3', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CptModifier3')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CptModifier4', 'transformation_label': 'String Indexer'}], 'feature': 'CptModifier4', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '12499', 'mean': '0.0', 'stddev': '0.0', 'min': '0000', 'max': '0000', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CptModifier4'}, {'feature_label': 'CptModifier4', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CptModifier4')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'pdiagnosis1', 'transformation_label': 'String Indexer'}], 'feature': 'pdiagnosis1', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '12499', 'mean': '', 'stddev': '', 'min': 'A04.72', 'max': 'Z98.890', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'pdiagnosis1'}, {'feature_label': 'pdiagnosis1', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('pdiagnosis1')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'pdiagnosis2', 'transformation_label': 'String Indexer'}], 'feature': 'pdiagnosis2', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '12499', 'mean': '', 'stddev': '', 'min': 'A04.72', 'max': 'Z99.89', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'pdiagnosis2'}, {'feature_label': 'pdiagnosis2', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('pdiagnosis2')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '12499', 'mean': '', 'stddev': '', 'min': 'F', 'max': 'M', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Gender'}, {'feature_label': 'Gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Gender')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DepartmentCategory', 'transformation_label': 'String Indexer'}], 'feature': 'DepartmentCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '12499', 'mean': '', 'stddev': '', 'min': 'CAT Scans', 'max': 'Urology Ultrasound', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DepartmentCategory'}, {'feature_label': 'DepartmentCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DepartmentCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SpecialityCategory', 'transformation_label': 'String Indexer'}], 'feature': 'SpecialityCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '12499', 'mean': '', 'stddev': '', 'min': 'Cardiac Surgery', 'max': 'Vascular Surgery', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SpecialityCategory'}, {'feature_label': 'SpecialityCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SpecialityCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PrimaryPayerCategory', 'transformation_label': 'String Indexer'}], 'feature': 'PrimaryPayerCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '12499', 'mean': '', 'stddev': '', 'min': 'Aetna US Healthcare', 'max': 'Work Comp', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PrimaryPayerCategory'}, {'feature_label': 'PrimaryPayerCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PrimaryPayerCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PrimaryPayerGroup', 'transformation_label': 'String Indexer'}], 'feature': 'PrimaryPayerGroup', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '12499', 'mean': '', 'stddev': '', 'min': 'Aetna', 'max': 'Work Comp', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PrimaryPayerGroup'}, {'feature_label': 'PrimaryPayerGroup', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PrimaryPayerGroup')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PrimaryPayerName', 'transformation_label': 'String Indexer'}], 'feature': 'PrimaryPayerName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '12499', 'mean': '', 'stddev': '', 'min': 'AARP Medicare Complete HMO', 'max': 'Work Comp', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PrimaryPayerName'}, {'feature_label': 'PrimaryPayerName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PrimaryPayerName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PrimaryPayerSubGroup1', 'transformation_label': 'String Indexer'}], 'feature': 'PrimaryPayerSubGroup1', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '12499', 'mean': '', 'stddev': '', 'min': 'Aetna', 'max': 'United Healthcare Medicare Advantage', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PrimaryPayerSubGroup1'}, {'feature_label': 'PrimaryPayerSubGroup1', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PrimaryPayerSubGroup1')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PrimaryPayerSubGroup2', 'transformation_label': 'String Indexer'}], 'feature': 'PrimaryPayerSubGroup2', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '12499', 'mean': '', 'stddev': '', 'min': 'Aetna', 'max': 'Workers Comp', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PrimaryPayerSubGroup2'}, {'feature_label': 'PrimaryPayerSubGroup2', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PrimaryPayerSubGroup2')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SecondPayerCategory', 'transformation_label': 'String Indexer'}], 'feature': 'SecondPayerCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '12499', 'mean': '', 'stddev': '', 'min': 'Aetna US Healthcare', 'max': 'United Healthcare', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SecondPayerCategory'}, {'feature_label': 'SecondPayerCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SecondPayerCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SecondPayerGroup', 'transformation_label': 'String Indexer'}], 'feature': 'SecondPayerGroup', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '12499', 'mean': '', 'stddev': '', 'min': 'Aetna', 'max': 'UHC', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SecondPayerGroup'}, {'feature_label': 'SecondPayerGroup', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SecondPayerGroup')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SecondPayerName', 'transformation_label': 'String Indexer'}], 'feature': 'SecondPayerName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '12499', 'mean': '', 'stddev': '', 'min': 'AARP 2ndary', 'max': 'Wellcare Staywell', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SecondPayerName'}, {'feature_label': 'SecondPayerName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SecondPayerName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SecondPayerSubGroup1', 'transformation_label': 'String Indexer'}], 'feature': 'SecondPayerSubGroup1', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '12499', 'mean': '', 'stddev': '', 'min': 'Aetna', 'max': 'United Healthcare Medicare Advantage', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SecondPayerSubGroup1'}, {'feature_label': 'SecondPayerSubGroup1', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SecondPayerSubGroup1')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SecondPayerSubGroup2', 'transformation_label': 'String Indexer'}], 'feature': 'SecondPayerSubGroup2', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '12499', 'mean': '', 'stddev': '', 'min': 'Aetna', 'max': 'Wellcare/Staywell', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SecondPayerSubGroup2'}, {'feature_label': 'SecondPayerSubGroup2', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SecondPayerSubGroup2')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'TeritoryPayerCategory', 'transformation_label': 'String Indexer'}], 'feature': 'TeritoryPayerCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '12499', 'mean': '', 'stddev': '', 'min': 'Aetna US Healthcare', 'max': 'United Healthcare', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'TeritoryPayerCategory'}, {'feature_label': 'TeritoryPayerCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('TeritoryPayerCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'TeritoryPayerGroup', 'transformation_label': 'String Indexer'}], 'feature': 'TeritoryPayerGroup', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '12499', 'mean': '', 'stddev': '', 'min': 'Aetna', 'max': 'UHC', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'TeritoryPayerGroup'}, {'feature_label': 'TeritoryPayerGroup', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('TeritoryPayerGroup')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'TeritoryPayerName', 'transformation_label': 'String Indexer'}], 'feature': 'TeritoryPayerName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '12499', 'mean': '', 'stddev': '', 'min': 'AARP 2ndary', 'max': 'Wellcare Staywell', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'TeritoryPayerName'}, {'feature_label': 'TeritoryPayerName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('TeritoryPayerName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'TeritoryPayerSubGroup1', 'transformation_label': 'String Indexer'}], 'feature': 'TeritoryPayerSubGroup1', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '12499', 'mean': '', 'stddev': '', 'min': 'Beechstreet', 'max': 'T_90', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'TeritoryPayerSubGroup1'}, {'feature_label': 'TeritoryPayerSubGroup1', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('TeritoryPayerSubGroup1')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'TeritoryPayerSubGroup2', 'transformation_label': 'String Indexer'}], 'feature': 'TeritoryPayerSubGroup2', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '12499', 'mean': '', 'stddev': '', 'min': 'Aetna', 'max': 'Wellcare/Staywell', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'TeritoryPayerSubGroup2'}, {'feature_label': 'TeritoryPayerSubGroup2', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('TeritoryPayerSubGroup2')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ServiceItemName', 'transformation_label': 'String Indexer'}], 'feature': 'ServiceItemName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '12499', 'mean': '148723.88', 'stddev': '728023.45', 'min': '10060', 'max': 'S9083', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ServiceItemName'}, {'feature_label': 'ServiceItemName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ServiceItemName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ServiceItemCategory', 'transformation_label': 'String Indexer'}], 'feature': 'ServiceItemCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '12499', 'mean': '', 'stddev': '', 'min': 'ADMIN ECG CONTRAST AGENT', 'max': 'Zinc paste band w >=3"<5"/yd', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ServiceItemCategory'}, {'feature_label': 'ServiceItemCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ServiceItemCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ServiceLocation', 'transformation_label': 'String Indexer'}], 'feature': 'ServiceLocation', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '12499', 'mean': '', 'stddev': '', 'min': 'Ambulatory Surgical Center', 'max': 'Urgent Care Facility', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ServiceLocation'}, {'feature_label': 'ServiceLocation', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ServiceLocation')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split


def randomforestregressor(df, labels, features):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    rf_model = RandomForestRegressor()
    X_train, X_test, y_train, y_test = train_test_split(
        df[features], df[labels])
    rf_model.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % rf_model.score(X_test, y_test))

    data = {'model': rf_model,
            'X_test': X_test,
            'y_test': y_test,
            'label': labels,
            'columnNames': df.columns}
    return data


***READING DATAFRAME***

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

#%run bceligdenialsdayspredHooks.ipynb
try:
	#sourcePreExecutionHook()

	sqlserver = RDBMSConnector.fetch(spark, "{'host': 'wsc-sqlserver1.database.windows.net', 'port': '1433', 'password': 'G5gTW5rnKVeAZWXm', 'database': 'NextGen_BOND_CLINIC', 'optionalDB': 'NextGen_BOND_CLINIC', 'user': 'Numtra', 'qtype': 'query', 'query': "SELECT DISTINCT \r\n\r\n\t\t\tsub2.ChargeDetailID, \r\n\t\t\tsub2.DateofService,sub2.TransactionDate,\r\n\t\t\tCASE WHEN Sub1.FirstReasonDate IS NOT NULL THEN DATEDIFF(day, Sub1.ClaimSubmitDate, Sub1.FirstReasonDate) ELSE 0 END as days_to_firstdenial,\r\n\r\n\t\t\tsub2.CPTCode,\r\n\t\t\tsub2.CptModifier,sub2.CptModifier2,sub2.CptModifier3,sub2.CptModifier4,\r\n\t\t\tsub2.pdiagnosis1,sub2.pdiagnosis2,\r\n\t\t\tsub2.IsBillable , \r\n\t\t\tsub2.Age, sub2.Gender,sub2.Amount,\r\n\t\t\tsub2.DepartmentCategory,sub2.SpecialityCategory,\r\n\t\t\tsub2.PrimaryPayerCategory ,sub2.PrimaryPayerGroup ,sub2.PrimaryPayerName ,sub2.PrimaryPayerSubGroup1 ,sub2.PrimaryPayerSubGroup2 ,\r\n\t\t\tsub2.SecondPayerCategory ,sub2.SecondPayerGroup ,sub2.SecondPayerName ,sub2.SecondPayerSubGroup1 ,sub2.SecondPayerSubGroup2 ,\r\n\t\t\tsub2.TeritoryPayerCategory ,sub2.TeritoryPayerGroup ,sub2.TeritoryPayerName ,sub2.TeritoryPayerSubGroup1 ,sub2.TeritoryPayerSubGroup2 ,\r\n\t\t\tsub2.ServiceItemName ,sub2.ServiceItemCategory,sub2.ServiceLocation\r\n\r\nFROM \r\n(\r\n\t\t\tSELECT\r\n\t\t\tDISTINCT\r\n\t\t\tcd.ChargeDetailID, \r\n\t\t\tcd.DateofService, TransactionDate,tr.ReasonDate,\r\n\t\t\tcptc.CPTCode,\r\n\t\t\tCASE WHEN cd.CPTModifier IS NOT NULL THEN cd.CPTModifier ELSE '0000' END as CptModifier,\r\n\t\t\tCASE WHEN cd.CPTModifier2 IS NOT NULL THEN cd.CPTModifier2 ELSE '0000' END as CptModifier2,\r\n\t\t\tCASE WHEN cd.CPTModifier3 IS NOT NULL THEN cd.CPTModifier3 ELSE '0000' END as CptModifier3,\r\n\t\t\tCASE WHEN cd.CPTModifier4 IS NOT NULL THEN cd.CPTModifier4 ELSE '0000' END as CptModifier4,\r\n\t\t\tdc.DiagnosisCode as pdiagnosis1,\r\n\t\t\tdc2.DiagnosisCode as pdiagnosis2,\r\n\t\t\t1 as IsBillable , \r\n\t\t\tFLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) AS Age, p.Gender,\r\n            cd.Amount,\r\n\t\t\td.DepartmentCategory,\r\n\t\t\ts.SpecialityCategory,\r\n\r\n\t\t\tCASE WHEN py.PayerCategory IS NOT NULL THEN py.PayerCategory ELSE 'NOT AVAILABLE' END as PrimaryPayerCategory ,\r\n\t\t\tCASE WHEN py.PayerGroup IS NOT NULL THEN py.PayerGroup ELSE 'NOT AVAILABLE' END as PrimaryPayerGroup ,\r\n\t\t\tCASE WHEN py.PayerName IS NOT NULL THEN py.PayerName ELSE 'NOT AVAILABLE' END as PrimaryPayerName ,\r\n\t\t\tCASE WHEN py.PayerSubGroup1 IS NOT NULL THEN py.PayerSubGroup1 ELSE 'NOT AVAILABLE' END as PrimaryPayerSubGroup1 ,\r\n\t\t\tCASE WHEN py.PayerSubGroup2 IS NOT NULL THEN py.PayerSubGroup2 ELSE 'NOT AVAILABLE' END as PrimaryPayerSubGroup2 ,\r\n\r\n\t\t\tCASE WHEN py1.PayerCategory IS NOT NULL THEN py1.PayerCategory ELSE 'NOT AVAILABLE' END as SecondPayerCategory ,\r\n\t\t\tCASE WHEN py1.PayerGroup IS NOT NULL THEN py1.PayerGroup ELSE 'NOT AVAILABLE' END as SecondPayerGroup ,\r\n\t\t\tCASE WHEN py1.PayerName IS NOT NULL THEN py1.PayerName ELSE 'NOT AVAILABLE' END as SecondPayerName ,\r\n\t\t\tCASE WHEN py1.PayerSubGroup1 IS NOT NULL THEN py1.PayerSubGroup1 ELSE 'NOT AVAILABLE' END as SecondPayerSubGroup1 ,\r\n\t\t\tCASE WHEN py1.PayerSubGroup2 IS NOT NULL THEN py1.PayerSubGroup2 ELSE 'NOT AVAILABLE' END as SecondPayerSubGroup2 ,\r\n\r\n\t\t\tCASE WHEN py2.PayerCategory IS NOT NULL THEN py2.PayerCategory ELSE 'NOT AVAILABLE' END as TeritoryPayerCategory ,\r\n\t\t\tCASE WHEN py2.PayerGroup IS NOT NULL THEN py2.PayerGroup ELSE 'NOT AVAILABLE' END as TeritoryPayerGroup ,\r\n\t\t\tCASE WHEN py2.PayerName IS NOT NULL THEN py2.PayerName ELSE 'NOT AVAILABLE' END as TeritoryPayerName ,\r\n\t\t\tCASE WHEN py2.PayerSubGroup1 IS NOT NULL THEN py2.PayerSubGroup1 ELSE 'NOT AVAILABLE' END as TeritoryPayerSubGroup1 ,\r\n\t\t\tCASE WHEN py2.PayerSubGroup2 IS NOT NULL THEN py2.PayerSubGroup2 ELSE 'NOT AVAILABLE' END as TeritoryPayerSubGroup2 ,\r\n\r\n\t\t\tCASE WHEN si.ServiceItemName IS NOT NULL THEN si.ServiceItemName ELSE 'NOT AVAILABLE' END as ServiceItemName ,\r\n\t\t\tCASE WHEN si.ServiceItemCategory IS NOT NULL THEN si.ServiceItemCategory ELSE 'NOT AVAILABLE' END as ServiceItemCategory,\r\n\t\t\tCASE WHEN sl.ServiceLocation IS NOT NULL THEN sl.ServiceLocation ELSE 'NOT AVAILABLE' END as ServiceLocation\r\n\r\n\r\nfrom ChargeDetail cd\r\ninner join Transactions t WITH (NOLOCK) on cd.ChargeDetailID = t.ChargeDetailID\r\ninner join Encounter e WITH (NOLOCK) on cd.DateOfService = e.DateOfService and cd.PatientID = e.PatientID  \r\nleft outer join AuthorizationsPA apa WITH (NOLOCK) on e.PatientID = apa.PatientID and apa.PayerId = t.PayerID \r\ninner join TransactionReason tr WITH (NOLOCK) on tr.ChargeDetailID = t.ChargeDetailID and tr.PayerId = t.PayerID\r\ninner join TransactionReasonCode trc WITH (NOLOCK) on trc.TransactionReasonCodeID = tr.TransactionReasonCodeID\r\ninner join Patient p WITH (NOLOCK) on p.PatientID = cd.PatientID\r\ninner join Department d WITH (NOLOCK) on d.DepartmentID = cd.DepartmentID\r\ninner join Provider pp WITH (NOLOCK) on pp.ProviderID = cd.ProviderID\r\ninner join Speciality s WITH (NOLOCK) on s.SpecialityID = pp.SpecialityID\r\ninner join DiagnosisCode dc WITH (NOLOCK) on dc.DiagnosisCodeID = cd.DiagnosisCodeID1\r\ninner join DiagnosisCode dc2 WITH (NOLOCK) on dc2.DiagnosisCodeID = cd.DiagnosisCodeID2\r\ninner join CptCode cptc WITH (NOLOCK) on cptc.CPTCodeID = cd.CPTCodeID\r\ninner join Payer py WITH (NOLOCK) on py.PayerID = t.PayerID\r\nleft join Payer py1 WITH (NOLOCK) on py1.PayerID = cd.SecondaryPayerID\r\nleft join Payer py2 WITH (NOLOCK) on py2.PayerID = cd.TertiaryPayerID\r\ninner join ServiceItem si WITH (NOLOCK) on cd.ServiceItemID = si.ServiceItemID\r\nleft outer join ServiceLocation sl WITH (NOLOCK) on cd.ServiceLocationID = sl.ServiceLocationID\r\ninner join ContractLinksPA clpa WITH (NOLOCK) on cd.PayerID = clpa.payer_id and cd.ProviderID = clpa.provider_id\r\ninner join ContractsPA copa WITH (NOLOCK) on clpa.contract_id = copa.contract_id\r\n\r\nWHERE cd.Amount >.01 and    e.IsBillable = 'true'\r\nAND CloseDate is not null and transactiontypeid=4\r\nAND TransactionDate between '1/01/2020' and '9/30/2020'\r\n) as sub2\r\n\r\nINNER JOIN\r\n(\r\n\t\tSELECT\r\n\t\t\t t.ChargeDetailID as cid, MIN(TransactionDate) as ClaimSubmitDate,\r\n\t\t\t MIN(tr1.Reasondate) as FirstReasonDate\r\n\t\tfrom Transactions t\r\n\t\tinner join Transactiontype tt on t.transactiontypeid = tt.TransactionTypeID\r\n\t\tinner join TransactionReason tr1 WITH (NOLOCK) on tr1.ChargeDetailID = t.ChargeDetailID\r\n\t\tinner join TransactionReasonCode trc WITH (NOLOCK) on trc.TransactionReasonCodeID = tr1.TransactionReasonCodeID\r\n\t\tWHERE t.Amount > 0.1 and trc.ReasonType!='Other'\r\n\t\tgroup by t.ChargeDetailID\r\n) as sub1\r\n\r\n on sub1.cid = sub2.ChargeDetailID \t\r\n Group by   \r\n \t\t\tsub2.ChargeDetailID,sub2.DateofService,sub2.TransactionDate,\r\n\t\t\tClaimSubmitDate,FirstReasonDate,\r\n\t\t\tsub2.CPTCode,sub2.CptModifier,sub2.CptModifier2,sub2.CptModifier3,sub2.CptModifier4,\r\n\t\t\tsub2.pdiagnosis1,sub2.pdiagnosis2,\r\n\t\t\tsub2.IsBillable , \r\n\t\t\tsub2.Age, sub2.Gender,sub2.Amount,\r\n\t\t\tsub2.DepartmentCategory,sub2.SpecialityCategory,\r\n\t\t\tsub2.PrimaryPayerCategory ,sub2.PrimaryPayerGroup ,sub2.PrimaryPayerName ,sub2.PrimaryPayerSubGroup1 ,sub2.PrimaryPayerSubGroup2 ,\r\n\t\t\tsub2.SecondPayerCategory ,sub2.SecondPayerGroup ,sub2.SecondPayerName ,sub2.SecondPayerSubGroup1 ,sub2.SecondPayerSubGroup2 ,\r\n\t\t\tsub2.TeritoryPayerCategory ,sub2.TeritoryPayerGroup ,sub2.TeritoryPayerName ,sub2.TeritoryPayerSubGroup1 ,sub2.TeritoryPayerSubGroup2 ,\r\n\t\t\tsub2.ServiceItemName ,sub2.ServiceItemCategory,sub2.ServiceLocation", 'dbtype': 'mssql', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/'}")
	#sourcePostExecutionHook(sqlserver)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run bceligdenialsdayspredHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(sqlserver,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "ChargeDetailID", "transformation_label": "String Indexer"}], "feature": "ChargeDetailID", "type": "string", "selected": "False", "replaceby": "max", "stats": {"count": "12499", "mean": "", "stddev": "", "min": "000328D9-2BD2-42A6-BA87-FB5443941796", "max": "FFF20D66-2565-4DA9-B73A-D929A09F02D5", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ChargeDetailID"}, {"transformationsData": [{"feature_label": "DateofService", "transformation_label": "Extract Date"}], "feature": "DateofService", "type": "date", "selected": "False", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "updatedLabel": "DateofService"}, {"transformationsData": [{"feature_label": "TransactionDate", "transformation_label": "Extract Date"}], "feature": "TransactionDate", "type": "date", "selected": "False", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "updatedLabel": "TransactionDate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "days_to_firstdenial", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "12499", "mean": "15.12", "stddev": "15.04", "min": "-3", "max": "368", "missing": "0"}, "updatedLabel": "days_to_firstdenial"}, {"transformationsData": [{"feature_label": "CPTCode", "transformation_label": "String Indexer"}], "feature": "CPTCode", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "12499", "mean": "145673.42", "stddev": "708211.55", "min": "10060", "max": "S9083", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CPTCode"}, {"transformationsData": [{"feature_label": "CptModifier", "transformation_label": "String Indexer"}], "feature": "CptModifier", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "12499", "mean": "8.92", "stddev": "19.43", "min": "0000", "max": "XS", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CptModifier"}, {"transformationsData": [{"feature_label": "CptModifier2", "transformation_label": "String Indexer"}], "feature": "CptModifier2", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "12499", "mean": "0.02", "stddev": "1.03", "min": "0000", "max": "RT", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CptModifier2"}, {"transformationsData": [{"feature_label": "CptModifier3", "transformation_label": "String Indexer"}], "feature": "CptModifier3", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "12499", "mean": "0.0", "stddev": "0.0", "min": "0000", "max": "RT", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CptModifier3"}, {"transformationsData": [{"feature_label": "CptModifier4", "transformation_label": "String Indexer"}], "feature": "CptModifier4", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "12499", "mean": "0.0", "stddev": "0.0", "min": "0000", "max": "0000", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CptModifier4"}, {"transformationsData": [{"feature_label": "pdiagnosis1", "transformation_label": "String Indexer"}], "feature": "pdiagnosis1", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "12499", "mean": "", "stddev": "", "min": "A04.72", "max": "Z98.890", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "pdiagnosis1"}, {"transformationsData": [{"feature_label": "pdiagnosis2", "transformation_label": "String Indexer"}], "feature": "pdiagnosis2", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "12499", "mean": "", "stddev": "", "min": "A04.72", "max": "Z99.89", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "pdiagnosis2"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "IsBillable", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "12499", "mean": "1.0", "stddev": "0.0", "min": "1", "max": "1", "missing": "0"}, "updatedLabel": "IsBillable"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Age", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "12499", "mean": "66.81", "stddev": "17.59", "min": "1", "max": "106", "missing": "0"}, "updatedLabel": "Age"}, {"transformationsData": [{"feature_label": "Gender", "transformation_label": "String Indexer"}], "feature": "Gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "12499", "mean": "", "stddev": "", "min": "F", "max": "M", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Gender"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Amount", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "12499", "mean": "208.99", "stddev": "368.26", "min": "0.0", "max": "11070.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Amount"}, {"transformationsData": [{"feature_label": "DepartmentCategory", "transformation_label": "String Indexer"}], "feature": "DepartmentCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "12499", "mean": "", "stddev": "", "min": "CAT Scans", "max": "Urology Ultrasound", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DepartmentCategory"}, {"transformationsData": [{"feature_label": "SpecialityCategory", "transformation_label": "String Indexer"}], "feature": "SpecialityCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "12499", "mean": "", "stddev": "", "min": "Cardiac Surgery", "max": "Vascular Surgery", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SpecialityCategory"}, {"transformationsData": [{"feature_label": "PrimaryPayerCategory", "transformation_label": "String Indexer"}], "feature": "PrimaryPayerCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "12499", "mean": "", "stddev": "", "min": "Aetna US Healthcare", "max": "Work Comp", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PrimaryPayerCategory"}, {"transformationsData": [{"feature_label": "PrimaryPayerGroup", "transformation_label": "String Indexer"}], "feature": "PrimaryPayerGroup", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "12499", "mean": "", "stddev": "", "min": "Aetna", "max": "Work Comp", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PrimaryPayerGroup"}, {"transformationsData": [{"feature_label": "PrimaryPayerName", "transformation_label": "String Indexer"}], "feature": "PrimaryPayerName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "12499", "mean": "", "stddev": "", "min": "AARP Medicare Complete HMO", "max": "Work Comp", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PrimaryPayerName"}, {"transformationsData": [{"feature_label": "PrimaryPayerSubGroup1", "transformation_label": "String Indexer"}], "feature": "PrimaryPayerSubGroup1", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "12499", "mean": "", "stddev": "", "min": "Aetna", "max": "United Healthcare Medicare Advantage", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PrimaryPayerSubGroup1"}, {"transformationsData": [{"feature_label": "PrimaryPayerSubGroup2", "transformation_label": "String Indexer"}], "feature": "PrimaryPayerSubGroup2", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "12499", "mean": "", "stddev": "", "min": "Aetna", "max": "Workers Comp", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PrimaryPayerSubGroup2"}, {"transformationsData": [{"feature_label": "SecondPayerCategory", "transformation_label": "String Indexer"}], "feature": "SecondPayerCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "12499", "mean": "", "stddev": "", "min": "Aetna US Healthcare", "max": "United Healthcare", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SecondPayerCategory"}, {"transformationsData": [{"feature_label": "SecondPayerGroup", "transformation_label": "String Indexer"}], "feature": "SecondPayerGroup", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "12499", "mean": "", "stddev": "", "min": "Aetna", "max": "UHC", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SecondPayerGroup"}, {"transformationsData": [{"feature_label": "SecondPayerName", "transformation_label": "String Indexer"}], "feature": "SecondPayerName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "12499", "mean": "", "stddev": "", "min": "AARP 2ndary", "max": "Wellcare Staywell", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SecondPayerName"}, {"transformationsData": [{"feature_label": "SecondPayerSubGroup1", "transformation_label": "String Indexer"}], "feature": "SecondPayerSubGroup1", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "12499", "mean": "", "stddev": "", "min": "Aetna", "max": "United Healthcare Medicare Advantage", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SecondPayerSubGroup1"}, {"transformationsData": [{"feature_label": "SecondPayerSubGroup2", "transformation_label": "String Indexer"}], "feature": "SecondPayerSubGroup2", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "12499", "mean": "", "stddev": "", "min": "Aetna", "max": "Wellcare/Staywell", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SecondPayerSubGroup2"}, {"transformationsData": [{"feature_label": "TeritoryPayerCategory", "transformation_label": "String Indexer"}], "feature": "TeritoryPayerCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "12499", "mean": "", "stddev": "", "min": "Aetna US Healthcare", "max": "United Healthcare", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "TeritoryPayerCategory"}, {"transformationsData": [{"feature_label": "TeritoryPayerGroup", "transformation_label": "String Indexer"}], "feature": "TeritoryPayerGroup", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "12499", "mean": "", "stddev": "", "min": "Aetna", "max": "UHC", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "TeritoryPayerGroup"}, {"transformationsData": [{"feature_label": "TeritoryPayerName", "transformation_label": "String Indexer"}], "feature": "TeritoryPayerName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "12499", "mean": "", "stddev": "", "min": "AARP 2ndary", "max": "Wellcare Staywell", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "TeritoryPayerName"}, {"transformationsData": [{"feature_label": "TeritoryPayerSubGroup1", "transformation_label": "String Indexer"}], "feature": "TeritoryPayerSubGroup1", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "12499", "mean": "", "stddev": "", "min": "Beechstreet", "max": "T_90", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "TeritoryPayerSubGroup1"}, {"transformationsData": [{"feature_label": "TeritoryPayerSubGroup2", "transformation_label": "String Indexer"}], "feature": "TeritoryPayerSubGroup2", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "12499", "mean": "", "stddev": "", "min": "Aetna", "max": "Wellcare/Staywell", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "TeritoryPayerSubGroup2"}, {"transformationsData": [{"feature_label": "ServiceItemName", "transformation_label": "String Indexer"}], "feature": "ServiceItemName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "12499", "mean": "148723.88", "stddev": "728023.45", "min": "10060", "max": "S9083", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ServiceItemName"}, {"transformationsData": [{"feature_label": "ServiceItemCategory", "transformation_label": "String Indexer"}], "feature": "ServiceItemCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "12499", "mean": "", "stddev": "", "min": "ADMIN ECG CONTRAST AGENT", "max": "Zinc paste band w >=3\"<5\"/yd", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ServiceItemCategory"}, {"transformationsData": [{"feature_label": "ServiceLocation", "transformation_label": "String Indexer"}], "feature": "ServiceLocation", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "12499", "mean": "", "stddev": "", "min": "Ambulatory Surgical Center", "max": "Urgent Care Facility", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ServiceLocation"}, {"feature": "CPTCode_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "12499", "mean": "17.33", "stddev": "39.17", "min": "0.0", "max": "323.0", "missing": "0"}, "updatedLabel": "CPTCode_stringindexer_tra..."}, {"feature": "CptModifier_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "12499", "mean": "0.92", "stddev": "3.03", "min": "0.0", "max": "34.0", "missing": "0"}, "updatedLabel": "CptModifier_stringindexer..."}, {"feature": "CptModifier2_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "12499", "mean": "0.01", "stddev": "0.19", "min": "0.0", "max": "10.0", "missing": "0"}, "updatedLabel": "CptModifier2_stringindexe..."}, {"feature": "CptModifier3_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "12499", "mean": "0.0", "stddev": "0.01", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "CptModifier3_stringindexe..."}, {"feature": "CptModifier4_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "12499", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "CptModifier4_stringindexe..."}, {"feature": "pdiagnosis1_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "12499", "mean": "197.55", "stddev": "336.18", "min": "0.0", "max": "1610.0", "missing": "0"}, "updatedLabel": "pdiagnosis1_stringindexer..."}, {"feature": "pdiagnosis2_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "12499", "mean": "231.37", "stddev": "378.63", "min": "0.0", "max": "1763.0", "missing": "0"}, "updatedLabel": "pdiagnosis2_stringindexer..."}, {"feature": "Gender_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "12499", "mean": "0.45", "stddev": "0.5", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Gender_stringindexer_tran..."}, {"feature": "DepartmentCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "12499", "mean": "2.72", "stddev": "4.83", "min": "0.0", "max": "45.0", "missing": "0"}, "updatedLabel": "DepartmentCategory_string..."}, {"feature": "SpecialityCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "12499", "mean": "3.64", "stddev": "4.71", "min": "0.0", "max": "23.0", "missing": "0"}, "updatedLabel": "SpecialityCategory_string..."}, {"feature": "PrimaryPayerCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "12499", "mean": "1.57", "stddev": "2.39", "min": "0.0", "max": "19.0", "missing": "0"}, "updatedLabel": "PrimaryPayerCategory_stri..."}, {"feature": "PrimaryPayerGroup_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "12499", "mean": "0.85", "stddev": "1.75", "min": "0.0", "max": "13.0", "missing": "0"}, "updatedLabel": "PrimaryPayerGroup_stringi..."}, {"feature": "PrimaryPayerName_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "12499", "mean": "7.01", "stddev": "10.03", "min": "0.0", "max": "79.0", "missing": "0"}, "updatedLabel": "PrimaryPayerName_stringin..."}, {"feature": "PrimaryPayerSubGroup1_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "12499", "mean": "1.01", "stddev": "1.88", "min": "0.0", "max": "20.0", "missing": "0"}, "updatedLabel": "PrimaryPayerSubGroup1_str..."}, {"feature": "PrimaryPayerSubGroup2_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "12499", "mean": "3.0", "stddev": "4.07", "min": "0.0", "max": "26.0", "missing": "0"}, "updatedLabel": "PrimaryPayerSubGroup2_str..."}, {"feature": "SecondPayerCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "12499", "mean": "1.11", "stddev": "2.08", "min": "0.0", "max": "18.0", "missing": "0"}, "updatedLabel": "SecondPayerCategory_strin..."}, {"feature": "SecondPayerGroup_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "12499", "mean": "1.02", "stddev": "1.8", "min": "0.0", "max": "12.0", "missing": "0"}, "updatedLabel": "SecondPayerGroup_stringin..."}, {"feature": "SecondPayerName_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "12499", "mean": "2.96", "stddev": "7.58", "min": "0.0", "max": "88.0", "missing": "0"}, "updatedLabel": "SecondPayerName_stringind..."}, {"feature": "SecondPayerSubGroup1_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "12499", "mean": "0.85", "stddev": "1.58", "min": "0.0", "max": "19.0", "missing": "0"}, "updatedLabel": "SecondPayerSubGroup1_stri..."}, {"feature": "SecondPayerSubGroup2_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "12499", "mean": "1.13", "stddev": "2.38", "min": "0.0", "max": "23.0", "missing": "0"}, "updatedLabel": "SecondPayerSubGroup2_stri..."}, {"feature": "TeritoryPayerCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "12499", "mean": "0.05", "stddev": "0.47", "min": "0.0", "max": "11.0", "missing": "0"}, "updatedLabel": "TeritoryPayerCategory_str..."}, {"feature": "TeritoryPayerGroup_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "12499", "mean": "0.05", "stddev": "0.43", "min": "0.0", "max": "10.0", "missing": "0"}, "updatedLabel": "TeritoryPayerGroup_string..."}, {"feature": "TeritoryPayerName_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "12499", "mean": "0.17", "stddev": "1.53", "min": "0.0", "max": "35.0", "missing": "0"}, "updatedLabel": "TeritoryPayerName_stringi..."}, {"feature": "TeritoryPayerSubGroup1_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "12499", "mean": "0.05", "stddev": "0.37", "min": "0.0", "max": "9.0", "missing": "0"}, "updatedLabel": "TeritoryPayerSubGroup1_st..."}, {"feature": "TeritoryPayerSubGroup2_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "12499", "mean": "0.06", "stddev": "0.6", "min": "0.0", "max": "15.0", "missing": "0"}, "updatedLabel": "TeritoryPayerSubGroup2_st..."}, {"feature": "ServiceItemName_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "12499", "mean": "17.25", "stddev": "38.92", "min": "0.0", "max": "329.0", "missing": "0"}, "updatedLabel": "ServiceItemName_stringind..."}, {"feature": "ServiceItemCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "12499", "mean": "12.54", "stddev": "32.5", "min": "0.0", "max": "287.0", "missing": "0"}, "updatedLabel": "ServiceItemCategory_strin..."}, {"feature": "ServiceLocation_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "12499", "mean": "0.13", "stddev": "0.58", "min": "0.0", "max": "9.0", "missing": "0"}, "updatedLabel": "ServiceLocation_stringind..."}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run bceligdenialsdayspredHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=randomforestregressor(autofe, ["CPTCode_stringindexer", "CptModifier_stringindexer", "CptModifier2_stringindexer", "CptModifier3_stringindexer", "CptModifier4_stringindexer", "pdiagnosis1_stringindexer", "pdiagnosis2_stringindexer", "IsBillable", "Age", "Gender_stringindexer", "Amount", "DepartmentCategory_stringindexer", "SpecialityCategory_stringindexer", "PrimaryPayerCategory_stringindexer", "PrimaryPayerGroup_stringindexer", "PrimaryPayerName_stringindexer", "PrimaryPayerSubGroup1_stringindexer", "PrimaryPayerSubGroup2_stringindexer", "SecondPayerCategory_stringindexer", "SecondPayerGroup_stringindexer", "SecondPayerName_stringindexer", "SecondPayerSubGroup1_stringindexer", "SecondPayerSubGroup2_stringindexer", "TeritoryPayerCategory_stringindexer", "TeritoryPayerGroup_stringindexer", "TeritoryPayerName_stringindexer", "TeritoryPayerSubGroup1_stringindexer", "TeritoryPayerSubGroup2_stringindexer", "ServiceItemName_stringindexer", "ServiceItemCategory_stringindexer", "ServiceLocation_stringindexer"], "days_to_firstdenial")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
